## Load Forecasters

In [1]:
import sys
from common.path_utils import get_src_path, get_data_path

sys.path.append(str(get_src_path()))

import os

os.environ["LOCAL_CACHE"] = ".forecaster_cache"


from common.datatypes import ForecastingQuestion
import json


## AI Forecasting Bot Template


This is a simple bot template that you can use to forecast in the Metaculus AI Benchmarking Warmup Contest. It is a single shot GPT prompt that you are encouraged to experiment with!

In order to run this notebook as is, you'll need to enter a few API keys (use the key icon on the left to input them):

- `METACULUS_TOKEN`: you can find your Metaculus token under your bot's user settings page: https://www.metaculus.com/accounts/settings/, or on the bot registration page where you created the account: https://www.metaculus.com/aib/
- `OPENAPI_API_KEY`: get one from OpenAIs page: https://platform.openai.com/settings/profile?tab=api-keys
- `PERPLEXITY_API_KEY` - used to search up-to-date information about the question. Get one from https://www.perplexity.ai/settings/api




In [6]:
# Make sure you have set these in the sidebar to the left, by pressing the key icon.

#from google.colab import userdata
import os
from dotenv import load_dotenv

# Load .env file if it exists
dotenv_path = os.path.dirname(os.path.abspath(os.getcwd()))
dotenv_path = os.path.join(dotenv_path, '.env')
#print(dotenv_path)
if os.path.exists(dotenv_path):
    print(dotenv_path)
    load_dotenv(dotenv_path)
METACULUS_TOKEN = os.environ.get('METACULUS_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
OPENROUTER_API_KEY = os.environ.get('OPENROUTER_API_KEY')
ANTHROPIC_API_KEY = os.environ.get('ANTHROPIC_KEY')
assert(METACULUS_TOKEN is not None)

/Users/ashen/Desktop/consistency-forecasting/.env


### ChatGPT Prompt

You can change the prompt below to experiment. Key parameters that you can include in your prompt are:

*   `{title}` The question itself
*   `{summary_report}` A up to date news compliation generated from Perplexity
*   `{background}` The background section of the Metaculus question. This comes from the `description` field on the question
*   `{fine_print}` The fine print section of the question
*   `{today}` Today's date. Remember that your bot doesn't know the date unless you tell it explicitly!


**IMPORTANT**: As you experiment with changing the prompt, be aware that the last number output by GPT will be used as the forecast probability. The last line in the template specifies that.


In [10]:
PROMPT_TEMPLATE = """
You are a professional forecaster interviewing for a job.
The interviewer is also a professional forecaster, with a strong track record of
accurate forecasts of the future. They will ask you a question, and your task is
to provide the most accurate forecast you can. To do this, you evaluate past data
and trends carefully, make use of comparison classes of similar events, take into
account base rates about how past events unfolded, and outline the best reasons
for and against any particular outcome. You know that great forecasters don't
just forecast according to the "vibe" of the question and the considerations.
Instead, they think about the question in a structured way, recording their
reasoning as they go, and they always consider multiple perspectives that
usually give different conclusions, which they reason about together.
You can't know the future, and the interviewer knows that, so you do not need
to hedge your uncertainty, you are simply trying to give the most accurate numbers
that will be evaluated when the events later unfold.

Your interview question is:
{title}

Your research assistant says:
{summary_report}

background:
{background}

fine_print:
{fine_print}

Today is {today}.

You write your rationale and give your final answer as: "Probability: ZZ%", 0-100
"""

## Some setup code

This section sets up some simple helper code you can use to get data about forecasting questions and to submit a prediction

In [13]:
#!pip install -qU openai
import datetime
import json
import os
import requests
import re

from openai import OpenAI

AUTH_HEADERS = {"headers": {"Authorization": f"Token {METACULUS_TOKEN}"}}
API_BASE_URL = "https://www.metaculus.com/api2"
WARMUP_TOURNAMENT_ID = 3294
SUBMIT_PREDICTION = False

def find_number_before_percent(s):
    # Use a regular expression to find all numbers followed by a '%'
    matches = re.findall(r'(\d+)%', s)
    if matches:
        # Return the last number found before a '%'
        return int(matches[-1])
    else:
        # Return None if no number found
        return None

def post_question_comment(question_id, comment_text):
    """
    Post a comment on the question page as the bot user.
    """

    response = requests.post(
        f"{API_BASE_URL}/comments/",
        json={
            "comment_text": comment_text,
            "submit_type": "N",
            "include_latest_prediction": True,
            "question": question_id,
        },
        **AUTH_HEADERS,
    )
    response.raise_for_status()

def post_question_prediction(question_id, prediction_percentage):
    """
    Post a prediction value (between 1 and 100) on the question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/predict/"
    response = requests.post(
        url,
        json={"prediction": float(prediction_percentage) / 100},
        **AUTH_HEADERS,
    )
    response.raise_for_status()


def get_question_details(question_id):
    """
    Get all details about a specific question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/"
    response = requests.get(
        url,
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    return json.loads(response.content)

def list_questions(tournament_id=WARMUP_TOURNAMENT_ID, offset=0, count=10):
    """
    List (all details) {count} questions from the {tournament_id}
    """
    url_qparams = {
        "limit": count,
        "offset": offset,
        "has_group": "false",
        "order_by": "-activity",
        "forecast_type": "binary",
        "project": tournament_id,
        "status": "active",
        "type": "forecast",
        "include_description": "true",
    }
    url = f"{API_BASE_URL}/questions/"
    response = requests.get(url, **AUTH_HEADERS, params=url_qparams)
    response.raise_for_status()
    data = json.loads(response.content)
    return data

def call_perplexity(query):
    url = "https://api.perplexity.ai/chat/completions"
    headers = {
        "accept": "application/json",
        "authorization": f"Bearer {PERPLEXITY_API_KEY}",
        "content-type": "application/json",
    }
    payload = {
        "model": "llama-3-sonar-large-32k-chat",
        "messages": [
            {
                "role": "system",
                "content": """
You are an assistant to a superforecaster.
The superforecaster will give you a question they intend to forecast on.
To be a great assistant, you generate a concise but detailed rundown of the most relevant news, including if the question would resolve Yes or No based on current information.
You do not produce forecasts yourself.
""",
            },
            {"role": "user", "content": query},
        ],
    }
    response = requests.post(url=url, json=payload, headers=headers)
    response.raise_for_status()
    content = response.json()["choices"][0]["message"]["content"]
    return content

def get_gpt_prediction(question_details):
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    client = OpenAI(api_key=OPENAI_API_KEY)

    title = question_details["title"]
    resolution_criteria = question_details["resolution_criteria"]
    background = question_details["description"]
    fine_print = question_details["fine_print"]

    # Comment this line to not use perplexity
    summary_report = call_perplexity(title)

    chat_completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {
            "role": "user",
            "content": PROMPT_TEMPLATE.format(
                title=title,
                summary_report=summary_report,
                today=today,
                background=background,
                fine_print=fine_print,
            )
        }
        ]
    )

    gpt_text = chat_completion.choices[0].message.content

    # Regular expression to find the number following 'Probability: '
    probability_match = find_number_before_percent(gpt_text)

    # Extract the number if a match is found
    probability = None
    if probability_match:
        probability = int(probability_match) # int(match.group(1))
        print(f"The extracted probability is: {probability}%")
        probability = min(max(probability, 1), 99) # To prevent extreme forecasts

    return probability, summary_report, gpt_text

## test post comment and prediction

In [16]:
#def post_question_comment(question_id, comment_text):
#def post_question_prediction(question_id, prediction_percentage)

In [30]:
open_id = '25704'
closed_id = 25703
opens_soon_id = 25706

In [46]:
try:
    post_question_prediction(opens_soon_id, 1)
    post_question_comment(opens_soon_id, 't1')
except Exception as e:
    print(e)


403 Client Error: Forbidden for url: https://www.metaculus.com/api2/questions/25706/predict/


In [123]:
%load_ext autoreload
%autoreload 2

from bot_scrape import fetch_question_details_metaculus
import uuid

def metaculus_to_jsonl(m_dict):   
    """
    
    wanted format
    
    {
    "id": "1e3bb941-9bbb-45c8-b70a-ad4e24820ea9", 
    "title": "Will Scotland hold an official, sanctioned referendum on independence before May 3, 2024?", 
    "body": "This question will resolve as **Yes** if there is a referendum held in Scotland regarding Scotland's independence from the United Kingdom, and this referendum is held before May 3, 2024.  This referendum may be an \"advisory\" referendum; that is, it is not necessary for the referendum to have any legally binding effect.\n\nIt shall not be deemed resolved by a \u201cwildcat\u201d or \u201cCatalan\u201d style of referendum where the UK government has declined permission for the vote, nor by a vote organised by civil society institutions. A referendum must be deemed to have the consent of the London government", 
    "resolution_date": "2024-05-02 00:00:00", 
    "question_type": "binary", 
    "data_source": "metaculus", 
    "url": "https://www.metaculus.com/questions/6369", 
    "metadata": {
        "topics": [], 
        "market_prob": 0.001, 
        "background_info": "Scotland is one of the UK\u2019s four constituent nations - and its politics are currently dominated by the [Scottish National Party](https://whatscotlandthinks.org/), a party whose core aim is to remove Scotland from the UK and become an independent country. \n\nThe SNP government in Edinburgh previously successfully negotiated with the UK-wide government for the right to hold a referendum, [which took place in 2014](https://en.wikipedia.org/wiki/2014_Scottish_independence_referendum). The \u201cNo\u201d side (\u201cBetter Together\u201d) won 55% to 45%. The SNP has, however, won every Scottish national election since that date.\n\nThe SNP wish to have a fresh vote - stating that Brexit has changed the terms of the argument. The UK government has, to date, refused to countenance permitting such a vote. Under the Scotland Act, the Edinburgh government does not have the power to hold one [without permission.](https://www.legislation.gov.uk/ukpga/1998/46/section/30)"
        }, 
        
        "resolution": false}
    
    
    """

    res = {
        "id": str(uuid.uuid4()), ##random or not?
        "title": m_dict['title'],
        'body': None,##resolution criteria
        "resolution_date": min(m_dict['close_time'],m_dict['resolve_time']),
        "question_type": m_dict['possibilities']['type'], 
        "data_source": 'metaculus', 
        "url": 'https://www.metaculus.com'.format(m_dict['page_url']),
        "metadata": {
            "topics": [], 
            "api_url": m_dict['url'], 
            "market_prob": None, #market prob
            "background_info": None## background info
        }, 
        "resolution": m_dict['resolution']
    }

    res = fetch_question_details_metaculus(res)
    return res
        
        

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## List questions test

In [50]:
qs = list_questions(tournament_id=WARMUP_TOURNAMENT_ID, offset=0, count=100)


In [186]:
print(qs.keys())

dict_keys(['next', 'previous', 'results'])


In [ ]:
sample_question = qs['results'][1]
sample_question

In [52]:
skip = 0

print(qs['previous'])
print(qs['next'])

if skip:
    pass
else:
    for i,q in enumerate(qs['results']):
        print(i)
        print(q)
        print('\n')


None
None
0
{'active_state': 'OPEN', 'url': 'https://www.metaculus.com/api2/questions/25704/', 'page_url': '/questions/25704/will-exactly-2-starship-launches-reach-low-earth-orbit-by-sept-30-2024/', 'id': 25704, 'author': 120279, 'author_name': 'Tom_Metaculus', 'author_id': 120279, 'title': 'Will exactly 2 Starship launches reach low-Earth orbit by Sept 30, 2024?', 'title_short': 'Will exactly 2 Starship launches reach l', 'group_label': '', 'resolution': None, 'resolved_option': None, 'created_time': '2024-06-27T01:15:10.203983Z', 'publish_time': '2024-06-30T00:00:00Z', 'close_time': '2024-07-01T00:00:00Z', 'effected_close_time': None, 'resolve_time': '2024-09-30T00:00:00Z', 'possibilities': {'type': 'binary'}, 'scoring': {}, 'type': 'forecast', 'user_perms': {'PREDICT': True, 'RESOLVE': False, 'COMMENT_READ': True, 'COMMENT_POST': True, 'COMMENT_EDIT': True, 'COMMENT_MOD': False, 'EDIT_DRAFT_CONTENT': False, 'EDIT_PENDING_CONTENT': False, 'EDIT_UPCOMING_CONTENT': False, 'EDIT_LIVE_CO

In [194]:
import asyncio


sample_question = qs['results'][1]


if 'ipykernel' in sys.modules:
    # We're in a Jupyter Notebook
    sample_question = await (metaculus_to_jsonl(sample_question))    # Add Jupyter-specific code here

else:
    # We're not in a Jupyter Notebook
    sample_question = asyncio.run(metaculus_to_jsonl(sample_question))    # Add Jupyter-specific code here

print(sample_question)
sample_question = ForecastingQuestion(**sample_question)


{'id': '61738b93-29fe-40df-94b0-07184d7e9414', 'title': 'Will exactly 2 Starship launches reach low-Earth orbit by Sept 30, 2024?', 'body': "This question will resolve as the number of SpaceX Starship launches that reach low-Earth orbit, defined as achieving an altitude of at least 160 kilometers (approximately 100 miles) above the Earth's surface, by Sept 30, 2024.\n\nThe resolution will be based on official mission reports from SpaceX, and may also consider data or reporting from aerospace monitoring organizations or authorities, as well as other credible sources such as international media outlets.", 'resolution_date': '2024-07-01T00:00:00Z', 'question_type': 'binary', 'data_source': 'metaculus', 'url': 'https://www.metaculus.com', 'metadata': {'topics': [], 'api_url': 'https://www.metaculus.com/api2/questions/25704/', 'market_prob': None, 'background_info': 'SpaceX\'s Starship is at the forefront of the next generation of spacecrafts, designed with the ambitious goals of enabling h

In [ ]:
if 'ipykernel' in sys.modules:
    # We're in a Jupyter Notebook
    final_prob = await (af.call_async(sentence=sample_question))
    
else:
    # We're not in a Jupyter Notebook
    final_prob = asyncio.run(af.call_async(sentence=sample_question))


In [140]:
final_prob

0.75

## Run and call forecaster

In [137]:

import logging

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)  # configure root logger

# %%
from forecasters.advanced_forecaster import AdvancedForecaster

af = AdvancedForecaster(
    MAX_WORDS_NEWSCATCHER=5,
    MAX_WORDS_GNEWS=8,
    SEARCH_QUERY_MODEL_NAME="gpt-4o",
    SUMMARIZATION_MODEL_NAME="gpt-4o",
    BASE_REASONING_MODEL_NAMES=["gpt-4o", "gpt-4o"],
    RANKING_MODEL_NAME="gpt-4o",
    AGGREGATION_MODEL_NAME="gpt-4o",
)


Loading AdvancedForecaster...
Overriding retrieval_config: MAX_WORDS_NEWSCATCHER=5
Overriding retrieval_config: MAX_WORDS_GNEWS=8
Overriding retrieval_config: SEARCH_QUERY_MODEL_NAME=gpt-4o
Overriding retrieval_config: SUMMARIZATION_MODEL_NAME=gpt-4o
Overriding retrieval_config: RANKING_MODEL_NAME=gpt-4o
Overriding reasoning_config: BASE_REASONING_MODEL_NAMES:=['gpt-4o', 'gpt-4o']
Overriding reasoning_config: AGGREGATION_MODEL_NAME:=gpt-4o
Initialized forecaster with settings:


## GPT prediction and submitting a forecast

This is an example of how you can use the helper functions from above.

In [ ]:

question_id = 25140
question_details = get_question_details(question_id)
# print(question_details)

prediction, perplexity_result, gpt_result = get_gpt_prediction(question_details)
print("GPT predicted: ", prediction, perplexity_result, gpt_result)


if prediction is not None and SUBMIT_PREDICTION:
    post_question_prediction(question_id, prediction)
    comment = "PERPLEXITY\n\n" + perplexity_result + "\n\n#########\n\n" + "GPT\n\n" + gpt_result
    post_question_comment(question_id, comment)


The extracted probability is: 25%
GPT predicted:  25 Here's a concise rundown of the most relevant news to help the superforecaster with this question:

**Current Situation:**

* The 2023-24 respiratory virus season is ongoing in the US, with COVID-19, influenza, and Respiratory Syncytial Virus (RSV) circulating simultaneously.
* According to the Centers for Disease Control and Prevention (CDC), the current weekly hospitalization rates for each virus are:
	+ COVID-19: 14.1 per 100,000 (as of February 11, 2023) [1]
	+ Influenza: 23.1 per 100,000 (as of February 11, 2023) [2]
	+ RSV: 2.4 per 100,000 (as of February 4, 2023) [3]
* The combined peak for each virus has not yet occurred, as the rates are still increasing or plateauing.

**Trends and Projections:**

* The CDC's FluView report suggests that influenza activity is expected to continue for several more weeks, with the peak potentially occurring in late February or early March [2].
* COVID-19 hospitalization rates have been increa